## Mempersiapkan alat

# Prediksi Penyakit Jantung Menggunakan Machine Learning

## 1. Definisi Masalah

Dalam kasus ini, masalah yang saya jelajahi adalah masalah klasifikasi biner.

Saya akan mencari informasi dan memprediksi apakah seseorang memiliki penyakit jantung atau tidak.

## 2. Data

Data asli berasal dari database Cleveland dari UCI Machine Learning Repository.

Database asli berisi 76 atribut, tetapi di sini hanya 14 atribut yang akan digunakan, atribut juga disebut fitur.

Atribut dan fitur juga disebut sebagai variabel bebas dan variabel terikat, saya menggunakan variabel bebas untuk memprediksi variabel terikat. 

Dalam data ini, variabel bebas adalah atribut informasi pasien dan variabel terikat adalah apakah mereka memiliki penyakit jantung atau tidak.

## 3. Evaluasi 

Matrik evaluasi adalah sesuatu yang biasanya ditentukan di awal proyek.

Karena pembelajaran mesin sangat eksperimental, jadi dalam proyek ini saya harus mendapatkan akurasi minimal 85%.

Karena sifatnya eksperimen, matrik evaluasi dapat berubah seiring waktu.


## 4. Fitur

Kamus data menjelaskan data yang saya hadapi. Seharusnya disini saya harus melakukan penelitian atau bertanya kepada ahli (seseorang yang tahu tentang data ini) untuk lebih lanjut.

Berikut ini adalah fitur yang akan saya gunakan untuk memprediksi variabel target (penyakit jantung atau tidak ada penyakit jantung).

1. age - Usia dalam tahun 
2. sex - (1 = male; 0 = female) 
3. cp - chest pain type 
    * 0: Typical angina: nyeri dada terkait mengurangi suplai darah ke jantung
    * 1: Atypical angina: nyeri dada tidak berhubungan dengan jantung
    * 2: Non-anginal pain: biasanya kejang esofagus (tidak berhubungan dengan jantung)
    * 3: Asymptomatic: nyeri dada tidak menunjukkan tanda-tanda penyakit
4. trestbps - mengistirahatkan tekanan darah (dalam mm Hg saat masuk ke rumah sakit
    * di atas 130-140 biasanya memprihatinkan
5. chol - serum cholestoral dalam mg/dl
    * serum = LDL + HDL + .2 * triglycerides
    * di atas 200 yang memprihatinkan
6. fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false) 
    * '>126' mg/dL signals diabetes
7. restecg - resting electrocardiographic results
    * 0: Tidak ada yang perlu diperhatikan
    * 1: ST-T Wave abnormality
        - dapat berkisar dari gejala ringan hingga masalah parah
        - sinyal detak jantung yang tidak normal
    * 2: Possible or definite left ventricular hypertrophy
        - Ruang pompa utama jantung yang diperbesar
8. thalach - denyut jantung maksimum tercapai
9. exang - latihan diinduksi angina (1 = yes; 0 = no) 
10. oldpeak - Depresi ST yang disebabkan oleh olahraga relatif terhadap istirahat 
    * melihat stres jantung saat berolahraga
    * jantung yang tidak sehat akan lebih stres
11. slope - kemiringan segmen ST latihan puncak
    * 0: Upsloping: detak jantung yang lebih baik dengan olahraga (tidak biasa)
    * 1: Flatsloping: perubahan minimal (jantung sehat yang khas)
    * 2: Downslopins: tanda-tanda jantung yang tidak sehat
12. ca - jumlah pembuluh darah utama (0-3) diwarnai oleh flourosopy
    * pembuluh berwarna berarti dokter dapat melihat darah yang melewatinya
    * semakin banyak gerakan darah semakin baik (tidak ada gumpalan)
13. thal - hasil stres thalium
    * 1,3: normal
    * 6: fixed defect: dulu cacat tapi sekarang baik-baik saja
    * 7: reversable defect: tidak ada gerakan darah yang tepat saat berolahraga 
14. target - memiliki penyakit atau tidak (1=yes, 0=no) (= atribut yang diprediksi)

In [ ]:
# EDA dan Plot library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# agar hasil visualisasi bisa langsung tercetak di Jupyter Notebook
%matplotlib inline

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Evaluasi Model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_roc_curve

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Load data

In [ ]:
df = pd.read_csv("../input/heart-disease/heart.csv")
df.shape

## Data Exploration (exploratory data analysis or EDA)

In [ ]:
df.head()

In [ ]:
df.tail()

## Frekuensi Jenis Kelamin

In [ ]:
# Cari tahu berapa banyak nilai dari target
df["target"].value_counts().sort_values(ascending=True)

In [ ]:
df["target"].value_counts().sort_values(ascending=True).plot(kind="bar", figsize=(10, 6), color=["salmon", "lightblue"]);

plt.title("Frekuensi Jenis Kelamin")
plt.xlabel("0 = Wanita, 1 = Pria")
plt.ylabel("Jumlah")
plt.xticks(rotation=0);


In [ ]:
# melihat informasi dataset
df.info()

In [ ]:
# melihat data yang hilang
df.isna().sum()

In [ ]:
df.describe()

## Frekuensi Penyakit Jantung dengan Jenis Kelamin

In [ ]:
df.sex.value_counts()

In [ ]:
# bandingkan kolom target dengan sex
pd.crosstab(df.target, df.sex)

In [ ]:
# membuat plot crosstab
pd.crosstab(df.target, df.sex).plot(kind="bar",
                                    figsize=(10, 6),
                                    color=["salmon", "lightblue"]);

plt.title("Frekuensi Penyakit Jantung dengan Jenis Kelamin");
plt.xlabel("0 = Tidak Sakit, 1 = Sakit");
plt.ylabel("Jumlah");
plt.legend(["Wanita", "Pria"]);
plt.xticks(rotation=0);

Karena ada 96 pasien wanita dan 72 diantaranya memiliki nilai positif penyakit jantung, jika pasiennya adalah seorang wanita, ada kemungkinan 75% dia menderita penyakit jantung.

Sedangkan untuk pasien pria, ada 207 hampir setengahnya 45% kemungkinan menunjukkan adanya penyakit jantung.

Jadi dapat disimpulkan berdasarkan grafik di atas, kemungkinan seseorang terkena penyakit jantung adalah 59%.

## Usia dan Denyut Jantung Maksimal

In [ ]:
plt.figure(figsize=(10, 6))

# scatter
plt.scatter(df.age[df.target==1],
            df.thalach[df.target==1],
            c="salmon")
            
plt.scatter(df.age[df.target==0],
            df.thalach[df.target==0],
            c="lightblue")
            
plt.title("Penyakit Jantung dalam Usia dan Denyut Jantung Maksimal")
plt.xlabel("Umur")
plt.ylabel("Denyut Jantung Maksimal")
plt.legend(["Sakit", "Tidak Sakit"]);

Pada scatter plot diatas tampak semakin muda seseorang, semakin tinggi detak jantung maksimum mereka.

In [ ]:
# cek distribusi kolom umur
df.age.plot.hist();

Ini adalah distribusi normal tetapi sedikit bergeser kekanan.

## Frekuensi Penyakit Jantung per Jenis Nyeri Dada

Mencoba variabel lain, kali ini nyeri dada (cp)

In [ ]:
pd.crosstab(df.cp, df.target)

In [ ]:
pd.crosstab(df.cp, df.target).plot(kind="bar",
                                   figsize=(10, 6),
                                   color=["lightblue", "salmon"])

plt.title("Frekuensi Penyakit Jantung per Jenis Nyeri Dada")
plt.xlabel("Jenis Nyeri Dada") 
plt.ylabel("Frekuensi")
plt.legend(["Tidak Sakit", "Sakit"])
plt.xticks(rotation=0);

**Kamus data**

- cp - chest pain type 
    * 0: Typical angina: nyeri dada terkait mengurangi suplai darah ke jantung
    * 1: Atypical angina: nyeri dada tidak berhubungan dengan jantung
    * 2: Non-anginal pain: biasanya kejang esofagus (tidak berhubungan dengan jantung)
    * 3: Asymptomatic: nyeri dada tidak menunjukkan tanda-tanda penyakit

Bagian yang menarik disini, Atypical Angina(1) menyatakan itu tidak berhubungan dengan jantung tetapi tampaknya memiliki rasio yang lebih tinggi daripada Typical Angina(0) yang berhubungan dengan jantung.

Jadi apa itu Atypical Angina?
Pada kasus seperti ini penting untuk diingat, apabila kamus data tidak memberi informasi yang cukup, kita perlu melakukan penelitian lebih lanjut tentang ini. Kita bisa bertanya kepada ahlinya, tetapi karena ini hanya proyek latihan, saya mencoba memahaminya dengan mencari di Google.

## Korelasi antara Variabel Bebas

Tabel korelasi memberikan gambaran tentang variabel mana yang mungkin atau tidak berdampak pada variabel target.

In [ ]:
corr_matrix = df.corr()
corr_matrix

In [ ]:
corr_matrix = df.corr()
plt.figure(figsize=(15,10))
sns.heatmap(corr_matrix,
            annot=True,
            linewidth=5,
            fmt=".2f",
            cmap="YlGnBu");

Koefisien korelasi bernilai negatif, berarti hubungan antara kedua variabel tersebut negatif atau saling berbanding terbalik. Koefisien korelasi bernilai positif, berarti hubungan antara kedua variabel tersebut positif atau saling berbanding lurus.

## Pemodelan

In [ ]:
df.head()

In [ ]:
# Variabel bebas
X = df.drop("target", axis=1)
X.head()

In [ ]:
# Variabel terikat
y = df.target.values
y

## Split Data

In [ ]:
np.random.seed(8)

Fungsi seed digunakan untuk menyimpan status fungsi random, sehingga dapat menghasilkan angka acak yang sama pada beberapa eksekusi kode pada mesin yang sama atau pada mesin yang berbeda (untuk nilai seed tertentu). Nilai seed adalah angka nilai sebelumnya yang dihasilkan oleh generator. Untuk pertama kalinya ketika tidak ada nilai sebelumnya, ia menggunakan waktu sistem saat ini.

In [ ]:
for i in range(5):
    np.random.seed(8)
    print(np.random.randint(1, 1000))

In [ ]:
## Split data train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train, len(y_train)

In [ ]:
y_test, len(y_test)

## Membandingkan Model Machine Learning

- Logistic Regression
- K-Nearest Neighbors
- Random Forest

In [ ]:
models = {
    "KNN": KNeighborsClassifier(),
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier()
}

def fit_and_score(models, X_train, X_test, y_train, y_test):
    """
    Menyesuaikan dan mengevaluasi model machine learning.
    models: dictionary dari model pembelajaran mesin Scikit-Learn yang berbeda.
    X_train: training data
    X_test: testing data
    y_train: label yang terkait dengan data pelatihan
    y_test: label yang terkait dengan data testing
    """
    np.random.seed(8)
    # Membuat dictionary untuk menyimpan skor model
    model_scores = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        model_scores[name] = model.score(X_test, y_test)
    return model_scores

## Menggunakan Parameter Default

In [ ]:
model_scores = fit_and_score(models, X_train, X_test, y_train, y_test)
model_scores

## Perbandingan Model

In [ ]:
model_compare = pd.DataFrame(model_scores, index=['accuracy'])
model_compare.T.plot.bar();

## Menggunakan Hyperparameter Tuning dan Cross Validation

### Tune Parameter KNN

Dengan mencoba beberapa nilai n_neighbors yang berbeda.

In [ ]:
# List train scores
train_scores = []

# List test scores
test_scores = []

# Mencoba beberapa nilai n
neighbors = range(1,21)

# algoritma
knn = KNeighborsClassifier()

# Loop
for i in neighbors:
    knn.set_params(n_neighbors = i)
    knn.fit(X_train, y_train)
    
    # Update scores
    train_scores.append(knn.score(X_train, y_train))
    test_scores.append(knn.score(X_test, y_test))

In [ ]:
train_scores

In [ ]:
test_scores

In [ ]:
plt.plot(neighbors, train_scores, label="Train score")
plt.plot(neighbors, test_scores, label="Test score")
plt.xticks(np.arange(1, 21, 2))
plt.xlabel("Number of neighbors")
plt.ylabel("Model score")
plt.ylim(ymin=0)
plt.legend()

print(f"Nilai KNN tertinggi pada data test: {max(test_scores)*100:.2f}%")

## RandomizedSearch()

In [ ]:
# Logistic Regression
log_reg_grid = {"C": np.logspace(-4, 4, 20),
                "solver": ["liblinear"]}

# RandomForestClassifier
rf_grid = {"n_estimators": np.arange(10, 1000, 50),
           "max_depth": [None, 3, 5, 10],
           "min_samples_split": np.arange(2, 20, 2),
           "min_samples_leaf": np.arange(1, 20, 2)}

In [ ]:
np.random.seed(8)

#LogisticRegression
rs_log_reg = RandomizedSearchCV(LogisticRegression(),
                                param_distributions=log_reg_grid,
                                cv=5,
                                n_iter=20,
                                verbose=True)

rs_log_reg.fit(X_train, y_train);

In [ ]:
rs_log_reg.best_params_

In [ ]:
rs_log_reg.score(X_test, y_test)

Setelah menggunakan RandomizedSearchCV() model Logistic Regression masih memiliki akurasi yang sama ketika menggunakan parameter default.

In [ ]:
np.random.seed(8)

#RandomForestClassifier
rs_rfc = RandomizedSearchCV(RandomForestClassifier(),
                            param_distributions=rf_grid,
                            cv=5,
                            n_iter=20,
                            verbose=True)

rs_rfc.fit(X_train, y_train);

In [ ]:
rs_rfc.best_params_

In [ ]:
rs_rfc.score(X_test, y_test)

Sedangkan model RandomForestClassifier akurasinya meningkat dari 85% menjadi 90% 

## GridSearchCV()

In [ ]:
np.random.seed(8)

#LogisticRegression
gs_log_reg = GridSearchCV(LogisticRegression(),
                                param_grid=log_reg_grid,
                                cv=5,
                                verbose=True)

gs_log_reg.fit(X_train, y_train);

In [ ]:
gs_log_reg.best_params_

In [ ]:
gs_log_reg.score(X_test, y_test)

Dalam hal ini, saya mendapatkan hasil yang sama seperti sebelumnya karena grid saya hanya memiliki maksimal 20 kombinasi hyperparameter yang berbeda.

**Note**: Saya tidak menggunakan Grid pada RandomForestClassifier karena akan memiliki banyak kombinasi yang berbeda dan membutuhkan waktu yang lama, karena ini hanya latihan dan menyingkat waktu saya menggunakan RandomizedSearchCV. 

## Evaluation Model Klasifikasi

In [ ]:
y_preds = gs_log_reg.predict(X_test)
y_preds

In [ ]:
y_test

## ROC dan AUC Scores

In [ ]:
plot_roc_curve(gs_log_reg, X_test, y_test);

## Confusion Matrix

In [ ]:
print(confusion_matrix(y_test, y_preds))

In [ ]:
sns.set(font_scale=1.5)

def plot_conf_mat(y_test, y_preds):
    """
    Plots a confusion matrix using Seaborn's heatmap().
    """
    fig, ax = plt.subplots(figsize=(3, 3))
    ax = sns.heatmap(confusion_matrix(y_test, y_preds),
                     annot=True, # Annotate the boxes
                     cbar=False)
    plt.xlabel("true label")
    plt.ylabel("predicted label")
    
plot_conf_mat(y_test, y_preds)

## Classification Report

In [ ]:
print(classification_report(y_test, y_preds))

In [ ]:
gs_log_reg.best_params_

In [ ]:
clf = LogisticRegression(C=1.623776739188721,
                         solver="liblinear")

In [ ]:
# Accuracy score
cv_acc = cross_val_score(clf,
                         X,
                         y,
                         cv=5, # 5-fold cross-validation
                         scoring="accuracy") # accuracy as scoring
cv_acc

In [ ]:
cv_acc = np.mean(cv_acc)
cv_acc

In [ ]:
# Precision score
cv_precision = np.mean(cross_val_score(clf,
                                       X,
                                       y,
                                       cv=5, # 5-fold cross-validation
                                       scoring="precision")) # precision as scoring
cv_precision

In [ ]:
# Recall score
cv_recall = np.mean(cross_val_score(clf,
                                    X,
                                    y,
                                    cv=5, # 5-fold cross-validation
                                    scoring="recall")) # recall as scoring
cv_recall

In [ ]:
# F1 score
cv_f1 = np.mean(cross_val_score(clf,
                                X,
                                y,
                                cv=5, # 5-fold cross-validation
                                scoring="f1")) # f1 as scoring
cv_f1

In [ ]:
# Visualizing
cv_metrics = pd.DataFrame({"Accuracy": cv_acc,
                            "Precision": cv_precision,
                            "Recall": cv_recall,
                            "F1": cv_f1},
                            index=[0])
cv_metrics.T.plot.bar(title="Cross-Validated Metrics", legend=False);